In [113]:
import itertools
import numpy as np
import pandas as pd
import math
import os

# Resin


In [114]:
# Reading and merging files
data_prices = [pd.read_csv(f'data/prices_round_1_day_{d}.csv', sep=';') for d in ('-2', '-1', '0')]
data_trades = [pd.read_csv(f'data/trades_round_1_day_{d}.csv', sep=';')[['timestamp', 'symbol', 'price', 'quantity']] for d in ('-2', '-1', '0')]

data_prices_resin = [df.loc[df['product'] == 'RAINFOREST_RESIN'] for df in data_prices]
data_trades_resin = [df.loc[df['symbol'] == 'RAINFOREST_RESIN'] for df in data_trades]


In [115]:
# Seting limit
limit_resin = 50
# 'Fair' value
fair_value_resin = 10000.0

In [143]:
def calculate_pnl(list_df_prices, list_df_trades, limit, correction):
    df_results = pd.DataFrame()
    for _ in range(len(list_df_prices)):
        df_prices = list_df_prices[_]
        df_trades = list_df_trades[_]

        # Initialising order params
        if _ == 0:
            df_prices['volume'] = 0
            df_prices['pnl'] = 0.0
            df_prices['ask'] = 0
            df_prices['ask_vol'] = 0
            df_prices['bid'] = 0
            df_prices['bid_vol'] = 0

            prev_vol = 0
            prev_pnl = 0.0
            prev_ask_vol = 0
            prev_bid_vol = 0
            prev_ask_price = 0
            prev_bid_price = 0
            
        for i, row in df_prices.iterrows():
            # Existing portfolio and PnL
            curr_vol = prev_vol
            curr_pnl = prev_pnl
            
            # Checking if our standing orders are triggered
            timestamp = row['timestamp']
            df_trades_timestamp = df_trades.loc[df_trades['timestamp'] == timestamp]
            
            for trade in range(len(df_trades_timestamp)):
                price = df_trades_timestamp.iloc[trade, :]['price']
                size = df_trades_timestamp.iloc[trade, :]['quantity']
                # Checking a new market buying order
                if price > prev_ask_price:
                    # Checking if there are any orders in the book better than ours (In that case that is sold and standing volume decreases)
                    while size > 0:
                        if row['ask_volume_1'] > 0:
                            if prev_ask_price >= row['ask_price_1']:
                                row['ask_volume_1'] -= min(row['ask_volume_1'], size)
                                size -= min(row['ask_volume_1'], size)
                            else:
                                break
                        elif row['ask_volume_2'] > 0:
                            if prev_ask_price >= row['ask_price_2']:
                                row['ask_volume_2'] -= min(row['ask_volume_2'], size)
                                size -= min(row['ask_volume_2'], size)
                            else:
                                break
                        elif row['ask_volume_3'] > 0:
                            if prev_ask_price >= row['ask_price_3']:
                                row['ask_volume_3'] -= min(row['ask_volume_3'], size)
                                size -= min(row['ask_volume_3'], size)
                            else:
                                break
                        # In case we have an existing best position (maybe because there aren't any other standing orders in book), match with order
                        else:
                            break
                    if prev_ask_vol > 0:
                        curr_pnl = curr_pnl + min(prev_ask_vol, size, curr_vol + limit_resin) * prev_ask_price
                        curr_vol = curr_vol - min(prev_ask_vol, size, curr_vol + limit_resin)
                        prev_ask_vol = max(0, prev_ask_vol - size)
                               
                                
                # Checking a new market selling order
                if price < prev_bid_price:
                    # Checking if there are any orders in the book better than ours (In that case that is sold and standing volume decreases)
                    while size > 0:
                        if row['bid_volume_1'] > 0:
                            if prev_bid_price <= row['bid_price_1']:
                                row['bid_volume_1'] -= min(row['bid_volume_1'], size)
                                size -= min(row['bid_volume_1'], size)
                            else:
                                break
                        elif row['bid_volume_2'] > 0:
                            if prev_bid_price <= row['bid_price_2']:
                                row['bid_volume_2'] -= min(row['bid_volume_2'], size)
                                size -= min(row['bid_volume_2'], size)
                            else:
                                break
                        elif row['bid_volume_3'] > 0:
                            if prev_bid_price <= row['bid_price_3']:
                                row['bid_volume_3'] -= min(row['bid_volume_3'], size)
                                size -= min(row['bid_volume_3'], size)
                            else:
                                break
                        # In case we have an existing best position (maybe because there aren't any other standing orders in book), match with order
                        else:
                            break
                    if prev_bid_vol > 0:
                        curr_pnl = curr_pnl - min(prev_bid_vol, size, abs(curr_vol - limit_resin) ) * prev_bid_price
                        curr_vol = curr_vol + min(prev_bid_vol, size, abs(curr_vol - limit_resin) )
                        prev_bid_vol = max(0, prev_bid_vol - size)
                        
            # Updating pnl and current (portfolio) volume
            df_prices.at[i, 'volume'] = curr_vol 
            df_prices.at[i, 'pnl'] = curr_pnl
            
            # Setting maximum possible volumes for our bid and ask
            curr_ask_vol = limit + curr_vol
            curr_bid_vol = limit - curr_vol
            
            df_prices.at[i, 'ask_vol'] = curr_ask_vol 
            df_prices.at[i, 'bid_vol'] = curr_bid_vol

            # Finding out our bid and ask, based on current best bid/asks in the book, the current Mid price, and the given correction 
            adjustment = round((fair_value_resin - row['mid_price']) * correction)

            curr_ask_price = row['ask_price_1'] + adjustment
            curr_bid_price = row['bid_price_1'] + adjustment

            df_prices.at[i, 'ask'] = curr_ask_price
            df_prices.at[i, 'bid'] = curr_bid_price
        
            # Update previous values for next iteration
            prev_vol = curr_vol
            prev_pnl = curr_pnl
            prev_ask_vol = curr_ask_vol
            prev_bid_vol = curr_bid_vol
            prev_ask_price = curr_ask_price
            prev_bid_price = curr_bid_price

        df_results = pd.concat([df_results, df_prices])

    final_pnl = df_results.iloc[-1, :]['pnl'] + df_results.iloc[-1, :]['volume'] * fair_value_resin
    return df_results, final_pnl
    

In [151]:
pnl_dict = {}
for adjustment in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]:  
    df_results, final_pnl = calculate_pnl(data_prices_resin, data_trades_resin, limit_resin, adjustment)
    pnl_dict[adjustment] = final_pnl
    df_results.to_csv(f'results/test_resin_correction_{str(adjustment)}.csv', index=False)  

C:\Users\david\AppData\Local\Temp\ipykernel_26704\3980008269.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices['volume'] = 0
C:\Users\david\AppData\Local\Temp\ipykernel_26704\3980008269.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices['pnl'] = 0.0
C:\Users\david\AppData\Local\Temp\ipykernel_26704\3980008269.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [ ]:
pnl_dict